# MSTL model

The notebook is structured as follows:
1. Overview of the modeling pipeline defined in the custom `heat_forecast.pipeline.mstl` module, which implements the MSTL model and its configuration.
2. Hyperparameter tuning through rolling-origin cross-validation.
3. Final evaluation on a held-out test set (a comparative analysis of model performance is provided in the separate notebook `Test comparison.ipynb`).


## Import libreries and data

You can run the notebook in two ways:

1. **Google Colab**: place the project folder `heat-forecast` in **MyDrive**. The setup cell below will mount Drive and automatically add `MyDrive/heat-forecast/src` to `sys.path` so `import heat_forecast` works out of the box.

2. **Local machine**:

   * **Installing our package:** from the project root, run `pip install -e .` once (editable install). Then you can open the notebook anywhere and import the package normally.
   * **Alternative:** if you’re running the notebook from `.../heat-forecast/notebooks/` without installing the package, the setup cell will detect `../src` and automatically add it to `sys.path`.

In [ ]:
# === Do not edit below ===
# --- Detect if running on Google Colab & Set base dir ---
# %cd /home/giovanni.lombardi/heat-forecast/notebooks
import subprocess
from pathlib import Path
import sys

def in_colab() -> bool:
    try:
        import google.colab  # type: ignore
        return True
    except Exception:
        return False

# Install required packages only if not already installed
def pip_install(pkg: str):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])

# Set base directory and handle environment
if in_colab():
    # Make sure IPython is modern (avoids the old %autoreload/imp issue if you ever use it)
    pip_install("ipython>=8.25")
    pip_install("ipykernel>=6.29")
    
    def install(package):
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

    for pkg in ["statsmodels", "statsforecast", "mlforecast"]:
        pip_install(pkg)

    # Mount Google Drive
    from google.colab import drive  # type: ignore
    drive.mount('/content/drive')

    # Set base directory to your Drive project folder
    BASE_DIR = Path('/content/drive/MyDrive/heat-forecast')

    # Add `src/` to sys.path for custom package imports
    SRC_PATH = BASE_DIR / 'src'
    if str(SRC_PATH) not in sys.path:
        sys.path.append(str(SRC_PATH))

    # Sanity checks (helpful error messages if path is wrong)
    assert SRC_PATH.exists(), f"Expected '{SRC_PATH}' to exist. Fix BASE_DIR."
    pkg_dir = SRC_PATH / "heat_forecast"
    assert pkg_dir.exists(), f"Expected '{pkg_dir}' package directory."
    init_file = pkg_dir / "__init__.py"
    assert init_file.exists(), f"Missing '{init_file}'. Add it so Python treats this as a package."

else:
    # Local: either rely on editable install (pip install -e .) or add src/ when running from repo
    # Assume notebook lives in PROJECT_ROOT/notebooks/
    BASE_DIR = Path.cwd().resolve().parent
    SRC_PATH = BASE_DIR / "src"

    added_src = False
    if (SRC_PATH / "heat_forecast").exists() and str(SRC_PATH) not in sys.path:
        sys.path.append(str(SRC_PATH))
        added_src = True

# --- Logging setup ---
import logging
from zoneinfo import ZoneInfo
from datetime import datetime

LOG_DIR  = (BASE_DIR / "logs")
LOG_DIR.mkdir(parents=True, exist_ok=True)
LOG_FILE = LOG_DIR / "run.log"
PREV_LOG = LOG_DIR / "run.prev.log"

# If there's a previous run.log with content, archive it to run.prev.log
if LOG_FILE.exists() and LOG_FILE.stat().st_size > 0:
    try:
        # Replace old run.prev.log if present
        if PREV_LOG.exists():
            PREV_LOG.unlink()
        LOG_FILE.rename(PREV_LOG)
    except Exception as e:
        # Fall back to truncating if rename fails (e.g., file locked)
        print(f"[warn] Could not archive previous log: {e}. Truncating current run.log.")
        LOG_FILE.write_text("")

# Configure logging: fresh file for this run + echo to notebook/stdout
file_handler   = logging.FileHandler(LOG_FILE, mode="w", encoding="utf-8")
stream_handler = logging.StreamHandler(sys.stdout)

fmt = logging.Formatter("%(asctime)s | %(levelname)s | %(name)s | %(message)s",
                        datefmt="%m-%d %H:%M:%S")
file_handler.setFormatter(fmt)
stream_handler.setFormatter(fmt)

root = logging.getLogger()
root.handlers[:] = [file_handler, stream_handler]  # replace handlers (important in notebooks)
root.setLevel(logging.INFO)

# Use Rome time
logging.Formatter.converter = lambda *args: datetime.now(ZoneInfo("Europe/Rome")).timetuple()

logging.captureWarnings(True)
logging.info("=== Logging started (fresh current run) ===")
logging.info("Previous run (if any): %s", PREV_LOG if PREV_LOG.exists() else "none")

if added_src:
    logging.info("heat_forecast not installed; added src/ to sys.path")
else:
    logging.info("heat_forecast imported without modifying sys.path (likely installed)")

OPTUNA_DIR = BASE_DIR / "results" / "finetuning" / "lstm"
OPTUNA_DIR.mkdir(parents=True, exist_ok=True)
logging.info("BASE_DIR (make sure it's '*/heat-forecast/', else cd and re-run): %s", BASE_DIR)
logging.info("LOG_DIR: %s", LOG_DIR)
logging.info("OPTUNA_DIR: %s", OPTUNA_DIR)

Ensure [compatibility with Numba](https://numba.readthedocs.io/en/stable/user/installing.html#numba-support-info).

In [ ]:
# === Do not edit below ===
import sys, numpy, numba
logging.info("=== Current Environment ===")
logging.info("Python : %s", sys.version.split()[0])
logging.info("NumPy  : %s", numpy.__version__)
logging.info("Numba  : %s", numba.__version__)

Imports:

In [ ]:
# === Do not edit below ===
%load_ext autoreload
%autoreload 2

# --- Standard Library ---
import logging
import warnings
import os
import stat
import shutil
from datetime import datetime
from functools import reduce
from itertools import product

# --- Scientific Stack ---
import pandas as pd
pd.set_option('display.float_format', '{:.3f}'.format)
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# --- Statsmodels ---
from statsmodels.tsa.seasonal import MSTL

# --- Nixtla ---
from statsforecast import StatsForecast
from statsforecast.models import (
    AutoETS, Naive, SeasonalNaive
)
from statsforecast.utils import ConformalIntervals

# --- Scikit-learn ---
from sklearn.model_selection import ParameterGrid

# --- Utility Libraries ---
from tqdm.notebook import tqdm
import yaml

# --- Custom package: heat_forecast ---
from heat_forecast.pipeline import MSTLPipeline, MSTLConfig

from heat_forecast.utils.cv_utils import (
    get_cv_params_v2, get_cv_params_for_test
)
from heat_forecast.utils.datasplit import generate_sets
from heat_forecast.utils.decomposition import (
    add_annual_component, decompose_annual_seasonality, remove_annual_component
)
from heat_forecast.utils.evaluation import (
    barplot_cv, custom_evaluate, 
    display_cv_summary, display_metrics, plot_cv_metric_by_cutoff
)
from heat_forecast.utils.plotting import (
    configure_time_axes, custom_plot_results, plot_cutoff_results, interactive_plot_cutoff_results
)
from heat_forecast.utils.transforms import (
    get_lambdas, make_transformer, transform_column
)

# --- Plot Config ---
plt.style.use("seaborn-v0_8")
mpl.rcParams.update({
    'font.size': 14,
    'axes.titlesize': 16,
    'axes.labelsize': 14,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 10,
    'figure.titlesize': 18,
    'axes.grid': True,
    'axes.grid.which': 'both',
})

# --- Cleanup Utility ---
from heat_forecast.utils.fileshandling import remove_tree

# --- YAML Customization ---
from heat_forecast.utils.yaml import safe_dump_yaml

logging.info("All imports successful.")

Import pre-elaborated data.

In [ ]:
heat_path = BASE_DIR / 'data' / 'timeseries_preprocessed' / 'heat.csv'
aux_path = BASE_DIR / 'data' / 'timeseries_preprocessed' / 'auxiliary.csv'
heat_df = pd.read_csv(heat_path, parse_dates=['ds'])
aux_df = pd.read_csv(aux_path, parse_dates=['ds'])

## Tutorial of the model's pipeline

Define the forecast horizon and split the data into training, validation, and test sets accordingly. For now, the validation period used for model tuning and selection is also referred to as the 'test set'. The true test set (the portion of data reserved exclusively for final evaluation) will cover the period from May 2024 to May 2025.

In [ ]:
horizon_type = 'week' # 'day' for one day horizon, 'week' for one week horizon
facilities = ['F1', 'F2', 'F3', 'F4', 'F5']  # Specify the facilities to use, e.g. ['F1', 'F2', ... ]

# === Do not edit below ===
if horizon_type == 'day':
    h = 24 
elif horizon_type == 'week':
    h = 24 * 7
else:
    raise ValueError("Unsupported horizon type. Use 'day' or 'week'.")

start_train = pd.to_datetime('2019-07-01') #'2019-07-01' is the fist date available
end_train   = pd.to_datetime('2024-01-18')
end_test    = end_train + pd.Timedelta(hours=h) # single validation window

heat_facilities_df = heat_df[heat_df['unique_id'].isin(facilities)].copy()
uids = heat_facilities_df['unique_id'].unique()
n_uids = len(uids)

heat_sets, _ = generate_sets(target_df=heat_facilities_df, 
                             start_train=start_train, 
                             end_train=end_train, 
                             end_test=end_test, 
                             aux_flag=False,
                             max_lag=0,
                             verbose=True)
heat_train_df, heat_test_df = heat_sets

### Apply a transformation to stabilize the variance of data

In [ ]:
# --- Configure transformation ---
transform  = "boxcox"   # choices: "boxcox", "log", "arcsinh", "arcsinh2" for arcsinh(./2), "arcsinh10", or "none"
lam_method = "loglik"   # only used if boxcox, choices: "guerrero", "loglik"
winter_focus = True     # only used if boxcox, whether to focus on winter season
season_length = 365     # Must be integer, used for guerrero only

# === Do not edit below ===
# --- Transform data ---
# pre‐compute lambdas per series if needed
if transform == "boxcox":
    lambdas = get_lambdas(
        heat_train_df, 
        method=lam_method, 
        winter_focus=winter_focus, 
        season_length=season_length,
        verbose=True
    )
    name = f"boxcox_{lam_method} with winter focus" if winter_focus else f"boxcox_{lam_method}"
else:
    lambdas = None
    name = transform if transform != "none" else "no transformation"

# apply the transformation
fwd = make_transformer(transform, 'y', lambdas, inv=False)
for df in (heat_train_df, heat_test_df):
    df["y_transformed"] = transform_column(df, fwd)

# --- Plot the tansformed data ---
fig, axes = plt.subplots(n_uids, 1, figsize=(15, n_uids*3))
if n_uids == 1:
    axes = [axes]
for uid, ax in zip(heat_train_df["unique_id"].unique(), axes):
    grp = heat_train_df.query("unique_id == @uid")
    sns.lineplot(data=grp, x="ds", y="y_transformed", ax=ax)
    ax.set_title(f"Series: {uid}")

configure_time_axes(axes, heat_train_df['ds'])

fig.suptitle('Heat Demand Transformed with: ' + name)
fig.supxlabel('Date')
fig.supylabel('Transformed Heat Demand')
fig.tight_layout(rect=[0.01, 0.01, 0.99, 0.99])

After applying Box–Cox or log/arcsinh transforms, we observe a handful of unusually low values, particularly in series F3, that deviate substantially from the mean. In principle, we could impute them (for example, with forward fill), but I’ve chosen to retain them for two reasons:

1. **Seasonal context:** These dips occur almost exclusively during the warmer months, and our forecasting models are specifically calibrated to prevent summer variability from bleeding into, and inflating, our winter forecasts.
2. **Built-in robustness of MSTL:** Robust MSTL inherently down-weights abrupt, isolated declines. Moreover, since these outliers occur mostly during summer, well ahead of the high-load period, even if some of them partially leak into the seasonal or trend components, the model typically has sufficient time to recalibrate before peak demand begins.

By leaving these summer anomalies intact, we leverage MSTL's outlier-resistance without introducing potential bias from imputation. 


### Extract annual seasonality on daily aggregated data

Directly applying MSTL to capture annual, weekly, and daily seasonalities at the hourly level is computationally expensive. To address this, we adopt a two-step decomposition strategy.

1. We extract annual seasonality from data aggregated at the daily level.
2. We then apply MSTL to the deseasonalized hourly series to isolate weekly and daily components only.

For extracting the annual seasonal pattern from the daily-aggregated data, we considered three methods:

* **Classical Decomposition (CD)**: the simplest approach, where the seasonal component is computed as the average of values separated by one year. This results in a fixed seasonal pattern, perfectly periodic with a yearly cycle.

* **STL** ([Cleveland et al., 1990](https://www.math.unm.edu/~lil/Stat581/STL.pdf)): a more flexible method that extracts seasonal and trend components iteratively using a combination of LOESS smoothing and moving averages. Unlike CD, STL allows the seasonal pattern to evolve slowly over time.

* **Robust STL** (also discussed in the same paper): an extension of STL designed to detect and down-weight outliers during the iterative decomposition process.

Initial experiments revealed that Robust STL is not suitable in our setting. Counterintuitively, the robust weighting tended to amplify the influence of outliers rather than suppress them. This likely stems from the limited number of annual cycles in our dataset (only five years of data) which reduces the algorithm's ability to identify and isolate outliers. Moreover, real-world anomalies such as unexpected cold spells can span multiple days, further complicating the model’s attempt to distinguish them from genuine seasonal structure. As a result, the algorithm mistakenly incorporated such irregular patterns into the seasonal component.

STL and CD performed similarly in initial comparisons. Given the limited number of cycles, we concluded that the added complexity of STL does not justify its use. In particular, STL requires careful tuning of multiple parameters (six in the original formulation and even more in `statsmodels`’ implementation). Considering the computational cost and tuning burden, we chose to proceed with Classical Decomposition for extracting annual seasonality.

To prevent annual seasonality from absorbing any weekly structure, we also experimented with an optional post-processing LOWESS smoothing step (`smooth_loess` flag). However, early trials showed that this additional smoothing offered no practical benefit. In fact, it occasionally masked real, sharp transitions between high and low demand periods. Consequently, we decided to exclude this step from the final procedure.

In [ ]:
decomposition_method = "CD"  # choices: "STL", "CD" (classical decomposition), "none"
robust_stl = True            # Set to True if you want to use robust STL decomposition (only used if decomposition_method is "STL")
smooth_loess = False         # Set to True if you want to smooth the seasonal component using LOWESS after its extraction 
window_loess = 7             # Window size for LOWESS smoothing (e.g. 7 days) (only used if smooth_loess is True)

# === Do not edit below ===
# --- Decompose the series ---
# aggregate to daily frequency
heat_daily_train_df = (
    heat_train_df
    .groupby(['unique_id', pd.Grouper(key='ds', freq='D')])
    .mean(numeric_only=True)
    .reset_index()  
)

# apply decomposition + additional smoothing of the seasonal component if requested
results_STL = decompose_annual_seasonality(
    decomposition_method=decomposition_method,
    df=heat_daily_train_df,
    target_col='y_transformed',
    robust=robust_stl,
    seasonal_STL=11,            # default seasonal window length for STL
    smooth_loess=smooth_loess,  
    window_loess=window_loess,
    data_in_dcmp=True           # Include the original data in the decomposition DataFrame
)

# --- Plot the decomposition ---
for uid, dcmp in results_STL.items():
    fig, axes = plt.subplots(4, 1, figsize=(12, 10), sharey=True)
    sns.lineplot(data=dcmp, x='ds', y='data',     ax=axes[0])
    sns.lineplot(data=dcmp, x='ds', y='trend',    ax=axes[1])
    sns.lineplot(data=dcmp, x='ds', y='seasonal', ax=axes[2])
    sns.lineplot(data=dcmp, x='ds', y='remainder',ax=axes[3])

    configure_time_axes(axes, dcmp['ds'])
    
    if decomposition_method == "STL":
        name = "STL Decomposition"
    elif decomposition_method == "CD":
        name = "Classical Decomposition"
    else:
        name = "No Decomposition"

    configure_time_axes(axes, heat_train_df['ds'])

    axes[0].set_ylabel('Data')
    axes[1].set_ylabel('Trend')
    axes[2].set_ylabel('Seasonal')
    axes[3].set_ylabel('Remainder')

    fig.suptitle(f"Series {uid}: {name}")
    fig.supxlabel('Date')
    fig.tight_layout(rect=[0.01, 0.01, 0.99, 0.99])

We observe the seasonal component capturing a modest dip at the end of December, which might be reflecting the impact of winter holidays.

### Remove annual seasonality from the data

Next, we subtract the annual seasonal component from the hourly data, yielding a deseasonalized series that we’ll use to extract daily and weekly patterns.

In [ ]:
# === Do not edit below ===
# ---  Remove the annual component from the training data ---
# Build a single DataFrame of the daily seasonal component
season_df = (
    pd.concat(
        [df[['ds','seasonal']].assign(unique_id=uid)
         for uid, df in results_STL.items()],
        ignore_index=True
    )
)

# Add a new column with the deseasonalized values
heat_train_df = remove_annual_component(
    df=heat_train_df,
    season_df=season_df,
    target_col='y_transformed',
    target_col_deseason='y_deseason_transformed'
)

# --- Plot the the deseasonalized data together with its daily average ---
fig, axes = plt.subplots(n_uids, 1, figsize=(15, 3*n_uids), sharex=True)
if n_uids == 1:
    axes = [axes]
for (id, group), ax in zip(heat_train_df.groupby('unique_id'), axes):
    sns.lineplot(data=group, x='ds', y='y_deseason_transformed', ax=ax, color='blue', alpha=0.5, label='Hourly')
    group_daily = (
        group
        .set_index(['ds'])
        .resample('D', level='ds')
        .mean(numeric_only=True)
        .reset_index()
    )
    sns.lineplot(data=group_daily, x='ds', y='y_deseason_transformed', ax=ax, color='black', label='Daily aggregate')
    ax.set_title(f'Deseasonalized Series: {id}')

configure_time_axes(axes, heat_train_df['ds'], global_legend=True, legend_fig=fig)
    
fig.suptitle('Per-ID Hourly & Daily Series with Annual Seasonality Removed')
fig.supxlabel('Date Time')
fig.tight_layout(rect=[0.01, 0.01, 0.99, 0.99])

We observe that during the transitions between summer and the extended-winter period (and vice versa), the deseasonalized series often shows marked deviations from its typical values. This happens because the transition between these two regimes in the original series does not always occur at exactly the same time each year. Unfortunately, addressing this issue is difficult without external variables, such as climatic exogenous inputs, that could help anticipate the regime change.

### Perform MSTL decomposition

In [ ]:
# --- Set params ---
train_horizon = 8*7*24          # (in hours) limit the data for MSTL decomposition (and subsequent forecasting) to the last `train_horizon` hours of the train set
periods_mstl = [24, 24 * 7]     # Periods for MSTL decomposition: 24 hours and 7 days (the two seasonalities)
windows_mstl = [11, 15]          # Choose the windows for the MSTL decomposition. Otherwise, the default is [11, 15] 
stl_kwargs_mstl = {             
    'robust': True              # Use robust MSTL decomposition
} 

# === Do not edit below ===
# --- Initialize training data and MSTL model ---
heat_train_MSTL_df = (
    heat_train_df
    .groupby('unique_id', group_keys=False)
    .tail(train_horizon)
)
MSTL_model = MSTL( 
    endog=heat_train_MSTL_df['y_deseason_transformed'],
    periods=periods_mstl,
    windows=windows_mstl,
    lmbda=None, # No Box-Cox transformation
    stl_kwargs=stl_kwargs_mstl
)

# --- Perform MSTL decomposition of each series separately ---
results_MSTL = {}
for uid, grp in heat_train_MSTL_df.groupby('unique_id'):
    # Sort & reset index
    grp = grp.sort_values('ds').reset_index(drop=True)
    
    # Fit MSTL
    res = MSTL( 
        endog=grp['y_deseason_transformed'],
        periods=periods_mstl,
        windows=windows_mstl,
        lmbda=None, # No Box-Cox transformation
        stl_kwargs=stl_kwargs_mstl
    ).fit()

    # Collect into a DataFrame
    dcmp = pd.DataFrame({
        'ds':       grp['ds'],
        'data':     grp['y_transformed'],
        'trend':    res.trend,
        'seasonal_24h': res.seasonal.iloc[:, 0],  # 24h seasonal component
        'seasonal_7d': res.seasonal.iloc[:, 1],   # 7d seasonal component
        'remainder': res.resid
    })
    results_MSTL[uid] = dcmp

# --- Plot the decomposition ---
for uid, dcmp in results_MSTL.items():
    fig, axes = plt.subplots(5, 1, figsize=(12, 10))
    color = 'black'
    sns.lineplot(data=dcmp, x='ds', y='data', ax=axes[0], color=color)
    sns.lineplot(data=dcmp, x='ds', y='trend', ax=axes[1], color=color)
    sns.lineplot(data=dcmp, x='ds', y='seasonal_24h', ax=axes[2], color=color)
    sns.lineplot(data=dcmp, x='ds', y='seasonal_7d', ax=axes[3], color=color)
    sns.lineplot(data=dcmp, x='ds', y='remainder',ax=axes[4], color=color)

    configure_time_axes(axes, heat_train_MSTL_df['ds'])

    axes[0].set_ylabel('Data')
    axes[1].set_ylabel('Trend')
    axes[2].set_ylabel('Seasonal 24h')
    axes[3].set_ylabel('Seasonal 7d')
    axes[4].set_ylabel('Remainder')

    fig.suptitle(f"Series {uid}: MSTL Decomposition for the Annual-Deseasonalized Series")
    fig.supxlabel('Date')
    fig.tight_layout(rect=[0.01, 0.01, 0.99, 0.99])


### Forecast trend+remainder with AutoETS, then add the seasonal components

In [ ]:
# --- Set params ---
trend_forecaster='AutoETS'                                          # choices: 'AutoETS' or 'Naive' for the trend forecaster
trend_forecaster_kwargs={
    'model': 'ZZN',                                                 # model specification
#   'prediction_intervals': ConformalIntervals(n_windows=4, h=h)    # type of prediction intervals to use
} 
level=[]                                                             # prediction intervals to compute

# === Do not edit below ===
# --- Forecast trend + remainder ---
logging.info("Forecasting with horizon: %d hours", h)

# Prepare the DataFrame with the trend + remainder
rows = []
for uid, dcmp in results_MSTL.items():
    y = dcmp['trend'] + dcmp['remainder']
    rows.append(pd.DataFrame({
        'unique_id': uid,
        'ds': dcmp['ds'],
        'y_trend_rem': y
    }))
y_df = pd.concat(rows, ignore_index=True)

# Forecast trend + remainder using the specified forecaster
if trend_forecaster == 'Naive':
    trend_model = Naive(alias='MSTL')
    sf_Naive = StatsForecast(
        models=[trend_model],
        freq='h',
    )
    trend_forecast_df = sf_Naive.forecast(h=h, df=y_df, target_col='y_trend_rem', level=level)
else:
    model_spec = trend_forecaster_kwargs.get('model', 'ZZN')
    pi_cust = trend_forecaster_kwargs.get('prediction_intervals',
                            ConformalIntervals(n_windows=4, h=h))
    trend_model = AutoETS(model=model_spec, prediction_intervals=pi_cust, alias='MSTL')
    sf_AutoETS = StatsForecast(
        models=[trend_model],
        freq='h',  # Frequency of the data
    )
    trend_forecast_df = sf_AutoETS.forecast(h=h, df=y_df, target_col='y_trend_rem', level=level) 

# --- Forecast seasonal components ---
# Build a DataFrame with the daily and weekly seasonal components
seas_df = pd.concat([
    pd.DataFrame({
        'unique_id': uid,
        'ds':         dcmp['ds'],
        'seasonal_24h': dcmp['seasonal_24h'],
        'seasonal_7d':  dcmp['seasonal_7d'],
    })
    for uid, dcmp in results_MSTL.items()
], ignore_index=True)

# Forecast the seasonal components with a naive model
naive_model24h = SeasonalNaive(season_length=24, alias='Naive24h')
sf_naive24h = StatsForecast(
    models=[naive_model24h], 
    freq='h'
)
naive24h_forecast_df = sf_naive24h.forecast(h, seas_df, target_col='seasonal_24h')
naive_model7d = SeasonalNaive(season_length=24*7, alias='Naive7d')
sf_naive7d = StatsForecast(
    models=[naive_model7d], 
    freq='h'
)
naive7d_forecast_df = sf_naive7d.forecast(h, seas_df, target_col='seasonal_7d')

# --- Add daily and weekly seasonalities to the trend+remainder forecast ---
# Merge the seasonal components into the trend + remainder forecast
trend_forecast_df = trend_forecast_df.merge(
    naive24h_forecast_df[['unique_id', 'ds', 'Naive24h']],
    on=['unique_id', 'ds'],
    how='left'
)
trend_forecast_df = trend_forecast_df.merge(
    naive7d_forecast_df[['unique_id', 'ds', 'Naive7d']],
    on=['unique_id', 'ds'],
    how='left'
)

# Select the columns to adjust
to_adjust = (
    trend_forecast_df
    .select_dtypes(include='number')  # all numeric columns
    .columns
    .difference(['unique_id', 'ds', 'Naive24h', 'Naive7d'])  # drop our merge‐keys / seasonal
)

# Adjust those columns by adding the seasonal components
trend_forecast_df[to_adjust] = (
    trend_forecast_df[to_adjust]
    .add(trend_forecast_df['Naive24h'], axis=0)  # add the 24h seasonal component
    .add(trend_forecast_df['Naive7d'], axis=0)   # add the 7d seasonal component
)

# Now drop the helper columns
trend_forecast_df.drop(columns=['Naive24h', 'Naive7d'], inplace=True)

# --- Add annual seasonal component back ---
# Build a DataFrame with the annual seasonal component
season_df = (
    pd.concat(
        [df[['ds','seasonal']].assign(unique_id=uid)
            for uid, df in results_STL.items()],
        ignore_index=True
    )
)

# Add to the forecast
forecast_df = add_annual_component(
    forecast_df=trend_forecast_df,
    season_df=season_df,
)

# --- Anti-transform the forecasts ---
to_transform = forecast_df.select_dtypes(include='number').columns

for c in to_transform:
    # Apply the transformation function to the forecasted values
    bwd = make_transformer(transform, c, lambdas, inv=True)
    forecast_df[c] = transform_column(forecast_df, bwd)

Plot the forecasts alongside predictions from a naive 24-hour baseline model that simply repeats the most recent 24-hour cycle.

In [ ]:
custom_plot_results(
    target_df=heat_df,
    forecast_df=forecast_df,
    start_offset=14 * 24,
    end_offset=3 * 24,
    with_naive=True,
    target_train_df=heat_train_df,
    order_of_models=['Naive24h', 'MSTL'],  # Order of models in the plot
    alpha=0.8,  # Transparency of the lines of the forecasts
    ids=facilities
)

### Evaluation

The `custom_evaluate` function computes evaluation metrics for the provided forecasts. If `with_naive=True`, it also generates naïve baseline forecasts on the fly for comparison. Additionally, if `with_pct_increase=True`, it calculates the percentage increase over the naïve baseline for each metric, appending a new column named `*_pct_inc`.

In [ ]:
evaluation_df = custom_evaluate(
    forecast_df=forecast_df,
    target_df=heat_df,
    metrics=['mae', 'rmse', 'mase', 'nmae', 'me'],
    with_naive=True,
    with_pct_increase=True
)
display_metrics(evaluation_df)

## Use of the custom MSTL Class

I've encapsulated that entire workflow in a custom class. Here’s how to invoke it to perform the same decomposition, modeling, and forecasting steps as above.

In [ ]:
# --- Create the MSTL configuration ---
# Include all parameters used for data transformation and fitting the MSTL model
config = MSTLConfig(
    transform=transform,
    lam_method=lam_method,
    winter_focus=winter_focus,
    season_length=season_length,
    decomposition_method=decomposition_method,
    smooth_loess=smooth_loess,
    window_loess=window_loess,
    robust_stl=robust_stl,
    train_horizon=train_horizon,
    windows_mstl=windows_mstl,
    stl_kwargs_mstl=stl_kwargs_mstl,
    periods_mstl=periods_mstl
)

# --- Instantiate the MSTL pipeline ---
mstl_pipeline = MSTLPipeline(
    target_df=heat_df,
    config=config,
)

# --- Fit the MSTL pipeline ---
mstl_pipeline.fit(
    start_train=start_train,
    end_train=end_train,
)

# --- Forecast with the MSTL pipeline ---
if trend_forecaster != 'AutoETS':
    logging.warning("The MSTL pipeline currently only supports AutoETS as trend forecaster. Overriding trend_forecaster to 'AutoETS'.")
mstl_forecast_df = mstl_pipeline.predict(
    h=h, 
    level=level, 
    trend_forecaster_kwargs=trend_forecaster_kwargs, 
        # The trend forecaster is AutoARIMA, the "naive" option in not implemented
)

# --- Plot the MSTL pipeline results ---
custom_plot_results(
    target_df=heat_df,
    forecast_df=mstl_forecast_df,
    start_offset=14 * 24,
    end_offset=3 * 24,
    with_naive=True,
    target_train_df=heat_train_df,
    order_of_models=['Naive24h', 'MSTL'],  # Order of models in the plot
    alpha=0.8,  # Transparency of the lines of the forecasts
    ids=facilities,
)
plt.show()

# --- Evaluate the MSTL pipeline forecasts ---
evaluation_df = custom_evaluate(
    forecast_df=mstl_forecast_df,
    target_df=heat_df,
    metrics=['mae', 'rmse', 'mase', 'nmae', 'me'],
    with_naive=True,
    with_pct_increase=True
)
display_metrics(evaluation_df)


Moreover, we can visualize the complete MSTL decomposition generated during model fitting.

In [ ]:
mstl_pipeline.plot_decomposition('F3') # Plot the decomposition for a specific facility, e.g. 'F3'

After a normal `.fit()`/`.predict()` cycle, you might need a forecast **from a later point in time** than the timestamp immediately after the end of the training data, **without refitting** the whole model. That’s exactly what `.forward` is for. It reuses everything learned at fit time (transforms, decomposition, and the ETS trend/remainder forecaster) and simply **re-anchors the seasonal components** to your new `context_end`. In other words, we change the *window we look back on*, not the model’s parameters.

Under the hood, `.forward`:

* Extends the stored daily/weekly seasonal components up to `context_end` by phase-aligned repetition.
* Rebuilds the **context target** $y_{\text{trend+rem}}$ from raw data by subtracting those re-anchored seasonals.
* Calls the cached ETS models’ **`.forward(...)`** on that context to produce the next `h` steps.
* Adds back the seasonal pieces and applies the inverse transform to return results on the original scale.


In [ ]:
context_end = end_train + pd.Timedelta(hours=24*7) # e.g. one week later

mstl_forecast_df = mstl_pipeline.forward(
    context_end=context_end,
    h=h, 
    level=level, 
    trend_forecaster_kwargs=trend_forecaster_kwargs, 
)

custom_plot_results(
    target_df=heat_df,
    forecast_df=mstl_forecast_df,
    start_offset=14 * 24,
    end_offset=3 * 24,
    with_naive=True,
    target_train_df=heat_train_df,
    order_of_models=['Naive24h', 'MSTL'],  # Order of models in the plot
    alpha=0.8,  # Transparency of the lines of the forecasts
    ids=['F3'],#facilities
)
plt.show()

### Cross validation

The `MSTLPipeline` class makes time-series cross-validation effortless via its `cross_validation` method, which mirrors the identically named routine in StatsForecast.

In [ ]:
run_cross_validation = True  # Set to False to skip cross-validation

# --- Set params ---
# Unique id(s) to use for cross-validation
facilities = ['F3']  # Specify a single ID to make cross-validation quicker
h = 24 * 7           # Forecast horizon (in hours)

# Params to set the cross-validation windows
max_n_fits = 5                                # Max number of fits to perform (to limit computation time)
step_size = 24 * 7                            # Step size between windows (in hours)
start_test_cv = pd.to_datetime('2023-12-10')  # Start of the first forecast horizon (before end_train+1h because we want many windows)
end_test_cv = pd.to_datetime('2024-03-10')    # (Upper bound for) End of the last forecast horizon

# === Do not edit below ===
if run_cross_validation:
    # Fitting params
    config = MSTLConfig()                         # MSTLConfig object with all the parameters used for MSTL fitting
    
    levels = []                                   # Prediction‐interval levels to compute, e.g. [80, 90]
    input_size = None                             # Size of each training window (in hours). If None, uses all data up to the cutoff.

    # --- Set CV windows ---
    # Get the cross-validation parameters to pass to cross_validation to use n_windows in the specified test_cv
    out = get_cv_params_v2(
        start_test_cv=start_test_cv,  
        end_test_cv=end_test_cv,                        
        step_size=step_size,
        max_n_fits=max_n_fits,               
        horizon_hours=h,
    )
    step_size = out['step_size']
    test_hours = out['test_hours']
    end_test_cv = out['end_test_actual']
    n_windows = out['n_windows']
    refit = out['refit']

    # --- Perform cross-validation with the naive model ---
    heat_facilities_df = heat_df[heat_df['unique_id'].isin(facilities)].copy()
    naive_model = SeasonalNaive(season_length=24, alias='Naive24h')
    sf = StatsForecast(models=[naive_model], freq='h')
    full_df = heat_facilities_df[heat_facilities_df['ds'] <= end_test_cv]  # Use all data up to the end of the last forecast horizon
    cv_naive = sf.cross_validation(
        h=h,
        df=full_df,
        n_windows=n_windows,
        step_size=step_size,    
        test_size=test_hours,   # Total size of the hold-out block for testing (from start_test_cv to end_test_cv)
        input_size=None,        # Use all data up to the cutoff
    )

    # --- Perform cross-validation with the MSTL model ---
    mstl_pipeline = MSTLPipeline(
        target_df=heat_facilities_df,
        config=config,
    )
    logging.info("Starting cross-validation...")
    cv_mstl = mstl_pipeline.cross_validation(
        h=h,
        test_size=test_hours,  
        end_test=end_test_cv,  
        step_size=step_size,  
        input_size=None,        # Size of each training window (in hours). If None, uses all data up to the cutoff.
        refit=refit,             # Whether to refit the model for each window
        verbose=True,           # Print progress
        alias='MSTL',           # Alias for the model in the results
    )
    logging.info("✓ Cross-validation completed.")

    # --- Merge results ---
    cv_df = cv_naive.merge(
        cv_mstl.drop(columns=['y']),
        on=['unique_id', 'ds', 'cutoff'],
        how='left',
    )

The `heat_forecast` package includes dedicated functions for visualizing and evaluating cross‑validation results.

Alongside reporting the usual cross-validation means and standard deviations of each metric across rolling forecast windows, I also include loss-difference statistics, defined for each cutoff $t$ and metric $m$ as:

$$
\text{LD-Candidate}_m(t) = \text{Candidate}_m(t) - \text{Naive}_{24h,m}(t),
$$

where $\text{Model}_m(t)$ denotes the value of metric $m$ (e.g., MAE, RMSE) computed over the forecast window $[t+1, t+h]$. In fact, we can replace $\text{Naive}_{24h,m}$ with any reference model we want.

Loss differences are particularly useful for comparing models because their variation reflects how consistently a candidate outperforms (or underperforms) the baseline, rather than just the absolute difficulty of the forecasting task. That is, the standard deviation of the LDs captures fluctuations in the relative performance gap between models across different forecast windows.

In contrast, examining the standard deviations of the raw metric values for each model (e.g., MAE or RMSE across rolling windows) is less informative for model comparison. Those standard deviations can be inflated by time-varying forecast difficulty. 

However, it's important to note that both LD standard deviations and raw metric standard deviations can be deflated by serial correlation across windows. This happens when model errors persist across time, making adjacent evaluations not fully independent.

In [ ]:
summary, combined_results = custom_evaluate_cv(
    cv_df=cv_df,
    metrics=['mae', 'rmse', 'mase', 'nmae'],  # List of metrics to compute
    target_df=heat_df,  # Training data mase
)
_ = display_cv_summary(summary)

In [ ]:
# Compute and summarise loss-difference (LD) statistics versus the baseline model.
summ_ld, combined_ld = compute_loss_diff_stats(combined_results)
_ = display_cv_summary(summ_ld)

In [ ]:
barplot_cv(combined_results)

In [ ]:
fig = plot_cv_metric_by_cutoff(
    combined_results=combined_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    figsize=(10,6)
)

In [ ]:
cutoffs = sorted(cv_df['cutoff'].unique())
fig = plot_cutoff_results(
    target_df=heat_df,
    cv_df=cv_df,
    cutoffs=cutoffs[:3],  # Plot first 3 cutoffs
    start_offset=24 * 3,
    end_offset=24 * 3,
)

After cross‑validation, MSTL retains the metadata from the most recent run, which you can retrieve like this:

In [ ]:
metadata = mstl_pipeline.last_cv_metadata
print(
    yaml.safe_dump(
        metadata,
        indent=4,              
    )
)

## MSTL fine tuning

In this section, we fine-tune the model's hyperparameters using a rolling-origin cross-validation procedure. The objective is to evaluate different parameter configurations across multiple forecast windows to identify those that yield the best performance. Fine-tuning is conducted independently for each time series (labeled F1 through F5) and for each forecast horizon (daily and weekly). In other words, the optimal hyperparameters are selected separately for each combination of series and forecast horizon.

Each configuration is evaluated over the historical period from October 2023, to the end of March 2024, a timeframe that covers most of the "cold regime" across all series for that year. For each forecast horizon, we define a sequence of evenly spaced rolling evaluation windows within this period:

- 30 windows for the daily forecast horizon;

- 20 windows for the weekly forecast horizon.

In each cross-validation fold, the model is trained on all data preceding the evaluation window (the training set), and evaluated on the window itself (the test set), using the error metrics described earlier. This process is repeated across all rolling windows and all hyperparameter combinations.

Finally, we compute the mean and standard deviation of the error metrics across all folds. These summary statistics are used to compare configurations and select the most robust model for each time series and forecast horizon.

As previously noted, the following settings were held fixed during the grid search:

* Annual seasonality decomposition was configured using `decomposition_method='CD'`, without final LOESS smoothing.
* Global MSTL settings (`stl_kwargs` in the `statsmodels` implementation) were kept at their default values, except for `robust=True`, which was enabled to improve resistance to outliers across all seasonal components.

Fine-tuning instead focused on the following hyperparameters:

* **`transform`**: the data transformation method, either a log transform or a Box-Cox transform with winter-focused training and log-likelihood-based lambda estimation.
* **`train_horizon`**: the number of data points retained for MSTL decomposition and subsequent forecasting. (Note: annual decomposition always uses the full training set).
* **`windows_mstl`**: the LOESS smoothing windows applied during MSTL for each seasonal component.


In [ ]:
finetune = False  # Set to False to skip fine-tuning

horizon_type = 'week' # 'day' for one day horizon, 'week' for one week horizon
id = 'F5'             # Specify a single ID for fine tuning

# === Do not edit below ===
if finetune:
    metadata = {}
    if horizon_type not in ['day', 'week']: 
        raise ValueError("Unsupported horizon type. Use 'day' or 'week'.")
    h = 24 * 7 if horizon_type == 'week' else 24
    heat_facilities_df = heat_df[heat_df['unique_id'] == id].copy()

    # --- Create directory for fine-tuning results ---
    timestamp = datetime.now().strftime("%Y%m%dT%H%M%S")
    run_id = f"{id}_{horizon_type}_finetuning_mstl_{timestamp}"
    path = BASE_DIR / "results" / "finetuning" / "mstl" / run_id
    metadata['run_id'] = run_id

    try:
        path.mkdir(parents=True, exist_ok=False)
        logging.info(f"Created directory for fine-tuning results: {path.relative_to(BASE_DIR)}")


        # --- Set params grid for fine tuning ---
        param_grid_1 = {
            'transform':              ['log'],
            'lam_method':             [None], 
            'winter_focus':           [None], 
            'train_horizon':          [7*7*24, 11*7*24, 15*7*24, 19*7*24, 23*7*24],
            'windows_mstl':           [[9, 11], [11, 15], [13, 17], [15, 19], [17, 21], [19, 23]], 
            'stl_kwargs_mstl':        [{'robust': True}],
        }
        param_grid_2 = {
            'transform':              ['boxcox'],
            'lam_method':             ['loglik'], 
            'winter_focus':           [True],
            'train_horizon':          [7*7*24, 11*7*24, 15*7*24, 19*7*24, 23*7*24],
            'windows_mstl':           [[9, 11], [11, 15], [13, 17], [15, 19], [17, 21], [19, 23]], 
            'stl_kwargs_mstl':        [{'robust': True}],
        }
        grid = list(ParameterGrid(param_grid_1))+list(ParameterGrid(param_grid_2))
        logging.info(f"Total parameter combinations: {len(grid)}")
        metadata['param_grid_1'] = param_grid_1
        metadata['param_grid_2'] = param_grid_2

        # --- Set windows with get_cv_params ---
        n_windows = 30 if horizon_type == 'day' else 20  # Number of windows for cross-validation
        start_test_cv = pd.to_datetime('2023-11-10')
        end_test_cv = pd.to_datetime('2024-04-10')  
        out = get_cv_params_v2(
            start_test_cv=pd.to_datetime('2023-11-10'),  
            end_test_cv=pd.to_datetime('2024-04-10'),    
            n_windows=n_windows,  
            horizon_hours=h,  
        )
        n_windows = out['n_windows'] 
        step_size = out['step_size']
        test_hours = out['test_hours']
        end_test_cv = out['end_test_actual']

        metadata['for_get_cv_params'] = {
            'n_windows': n_windows,
            'start_test_cv': str(start_test_cv),
            'end_test_cv': str(end_test_cv),
        }

        # --- Run CV once with the naive model for comparison ---
        params_placeholder = {key: None for key in param_grid_1.keys()}
        naive_model = SeasonalNaive(season_length=24, alias='Naive24h')
        sf = StatsForecast(models=[naive_model], freq='h')
        full_df = heat_facilities_df[heat_facilities_df['ds'] <= end_test_cv]  # Use all data up to the end of the last forecast horizon
        t0 = pd.Timestamp.now()
        cv_naive = sf.cross_validation(
            h=h,
            df=full_df,
            n_windows=n_windows,
            step_size=step_size,  
            test_size=test_hours,  
            input_size=None,  
            refit=True,  
        )
        t1 = pd.Timestamp.now()
        elapsed = (t1 - t0).total_seconds()
        record = {
            'name': 'Naive24h',
            'avg_elapsed_sec': elapsed/n_windows,  
            **params_placeholder,
        }

        # store results and corresponding params / name of the model / elapsed time
        records = [record]  # Initialize results with the naive model record
        cv_frames = [cv_naive]

        # --- Run grid search for MSTL model ---
        t_strt_cv = pd.Timestamp.now()
        for i, params in enumerate(tqdm(grid, desc="Grid search", leave=True)):
            mstlconfig = MSTLConfig(**params)
            mstlpipeline = MSTLPipeline(target_df=heat_facilities_df, config=mstlconfig)

            t0 = pd.Timestamp.now()
            cv_mstl = mstlpipeline.cross_validation(
                h=h,
                test_size=test_hours,  
                end_test=end_test_cv,  
                step_size=step_size,  
                input_size=None, 
                verbose=False,
                alias=f'MSTL_{i}',  
            )
            t1 = pd.Timestamp.now()
            elapsed = (t1 - t0).total_seconds()
            cv_mstl.drop(columns=['y'], inplace=True)
            cv_frames.append(cv_mstl)
            
            record = {
                'name': f'MSTL_{i}',
                **params,   
                'avg_elapsed_sec': elapsed/n_windows,
            }
            records.append(record)

        # --- Combine all cross-validation results into a single DataFrame ---
        key_cols = ["unique_id", "ds", "cutoff"]
        cv_results_df = reduce(
            lambda left, right: pd.merge(left, right, on=key_cols, how="outer"),
            cv_frames            
        )
        records_df = pd.DataFrame(records)
        results = (records_df, cv_results_df)

        t_end_cv = pd.Timestamp.now()
        elapsed_cv = str(t_end_cv - t_strt_cv)
        metadata['total_elapsed_time'] = elapsed_cv

        # --- Final save ---
        records_df.to_parquet(path / "records.parquet", compression="snappy")
        cv_results_df.to_parquet(path / "cv_results.parquet", compression="snappy")

        metadata_path = path / 'metadata.yaml'
        with open(metadata_path, 'w') as f:
            yaml.safe_dump(
                metadata,
                f,
                default_flow_style=False,    # block style
                sort_keys=False              # preserve insertion order
            )

        logging.info("✓ All artifacts saved successfully, fine-tuning completed.")

    except KeyboardInterrupt:
        logging.warning("✗ Interrupted; cleaning up.")
        remove_tree(path, require_within=BASE_DIR)
        logging.info("✓ Removed %s", Path(path).relative_to(BASE_DIR) if BASE_DIR in Path(path).resolve().parents else path)
        raise
    except Exception:
        logging.exception("✗ Error during test for id=%s, horizon=%s; cleaning up.", id, horizon_type)
        remove_tree(path, require_within=BASE_DIR)
        logging.info("✓ Removed %s", Path(path).relative_to(BASE_DIR) if BASE_DIR in Path(path).resolve().parents else path)
        raise

    logging.info(f"✓ Tuning completed.")


In [ ]:
# Optional: pick a run_id to analyze 
run_id = 'F5_week_finetuning_mstl_20250719T125709'  

# === Do not edit below ===
# --- Load the metadata for the run_id ---
path = BASE_DIR / "results" / "finetuning" / "mstl" / run_id
records_df = pd.read_parquet(path / "records.parquet")
cv_results_df = pd.read_parquet(path / "cv_results.parquet")

from heat_forecast.utils.evaluation import (
    evaluate_cv_forecasts, cv_evaluation_summary,
    compute_loss_diffs, display_cv_summary
)
# --- Evaluate results ---
with warnings.catch_warnings():
    warnings.filterwarnings(
        "ignore",
        message="DataFrame is highly fragmented",
        category=pd.errors.PerformanceWarning,
    )
    all_results = evaluate_cv_forecasts(
        cv_df=cv_results_df,
        target_df=heat_df, 
        metrics=['mae', 'rmse', 'smape'],  
    )
    summary = cv_evaluation_summary(all_results) 

In [ ]:
# --- Display with optional sorting ---
ws = display_cv_summary(summary, sort_metric='mae')

In [ ]:
# --- Display with optional sorting ---
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='Naive24h'
)
summary_ld = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(summary_ld, are_loss_diffs=True, sort_metric='mae')

In [ ]:
# --- Display the records of the first n models (based on the previous sorting) ---
n = 50     # how many?  

# === Do not edit below ===
# Select the first n models based on the sorted summary snd the corresponding records
s = wide_summary.reset_index()    
models_chosen = s.iloc[:n, s.columns.get_loc("model")].to_numpy().reshape(-1).tolist()
model_order = {model: i for i, model in enumerate(models_chosen)}
filtered_df = records_df.loc[records_df['name'].isin(models_chosen), :].copy()
filtered_df['order'] = filtered_df['name'].map(model_order)
ordered_df = filtered_df.sort_values('order').drop('order', axis=1).reset_index(drop=True)

# Display the ordered DataFrame with all rows and columns
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(ordered_df)


It can be useful to explore in more detail the results of the best models:

In [ ]:
n_models = 3  # how many?

# === Do not edit below ===
def best_mstl_models_plus_naive(ordered_df, n=3):
    """Get the best n mstl models based on the ordered DataFrame, plus the Naive24h model."""
    best_mstl_models = [m for m in ordered_df.loc[:n+1, 'name'] if m.startswith('MSTL_')][:n]
    return best_mstl_models + ['Naive24h']  

models = best_mstl_models_plus_naive(ordered_df, n=n_models)

fig = plot_cv_metric_by_cutoff(
    combined_results=combined_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=models,
)

#### Selected parameters


**Best parameters:**

In [ ]:
# === Do not edit below ===
mstl_optimal_params = {
    'F1': {
        'day': { # MSTL_59 from run_id = F1_day_finetuning_mstl_20250719T144130
            'decomposition_method': 'CD',
            'transform': 'boxcox',
            'lam_method': 'loglik',
            'winter_focus': True,
            'train_horizon': 23*7*24,
            'windows_mstl': [19, 23],
            'stl_kwargs_mstl': {'robust': True},
        },
        'week': { # MSTL_59 from run_id = F1_week_finetuning_mstl_20250719T143432
            'decomposition_method': 'CD',
            'transform': 'boxcox',
            'lam_method': 'loglik',
            'winter_focus': True,
            'train_horizon': 23*7*24,
            'windows_mstl': [19, 23],
            'stl_kwargs_mstl': {'robust': True},
        },
    },
    'F2': {
        'day': { # MSTL_44 from run_id = F2_day_finetuning_mstl_20250716T113343
            'decomposition_method': 'CD',
            'transform': 'boxcox',
            'lam_method': 'loglik',
            'winter_focus': True,
            'train_horizon': 15*7*24,
            'windows_mstl': [17, 21],
            'stl_kwargs_mstl': {'robust': True},
        },
        'week': { # MSTL_38 from run_id = F2_week_finetuning_mstl_20250716T143353
            'decomposition_method': 'CD',
            'transform': 'boxcox',	
            'lam_method': 'loglik',	
            'winter_focus': True,	
            'train_horizon': 11*7*24,	
            'windows_mstl': [13, 17],	
            'stl_kwargs_mstl': {'robust': True},	
        }
    },
    'F3': {
        'day': { # MSTL_29 from run_id = F3_day_finetuning_mstl_20250717T084909
            'decomposition_method': 'CD',
            'transform': 'log',
            'lam_method': None,
            'winter_focus': None,
            'train_horizon': 23*7*24,
            'windows_mstl': [19, 23],
            'stl_kwargs_mstl': {'robust': True},
        },
        'week': { # MSTL_29 from run_id = F3_week_finetuning_mstl_20250717T001618
            'decomposition_method': 'CD',
            'transform': 'log',
            'lam_method': None,
            'winter_focus': None,
            'train_horizon': 23*7*24,
            'windows_mstl': [19, 23],
            'stl_kwargs_mstl': {'robust': True},
        }
    },
    'F4': {
        'day': { # MSTL_23 from run_id = F4_day_finetuning_mstl_20250717T120310
            'decomposition_method': 'CD',
            'transform': 'log',
            'lam_method': None,
            'winter_focus': None,
            'train_horizon': 19*7*24,
            'windows_mstl': [19, 23],
            'stl_kwargs_mstl': {'robust': True},
        },
        'week': { # MSTL_47 from run_id = F4_week_finetuning_mstl_20250719T105932
            'decomposition_method': 'CD',
            'transform': 'boxcox',
            'lam_method': 'loglik',
            'winter_focus': True,
            'train_horizon': 15*7*24,
            'windows_mstl': [19, 23],
            'stl_kwargs_mstl': {'robust': True},
        },
    },
    'F5': {
        'day': { # MSTL_38 from run_id = F5_day_finetuning_mstl_20250718T002503
            'decomposition_method': 'CD',
            'transform': 'boxcox',
            'lam_method': 'loglik',
            'winter_focus': True,
            'train_horizon': 11*7*24,
            'windows_mstl': [13, 17],
            'stl_kwargs_mstl': {'robust': True},
        },
        'week': { # MSTL_26 from run_id = F5_week_finetuning_mstl_20250719T125709
            'decomposition_method': 'CD',
            'transform': 'log',
            'lam_method': None,
            'winter_focus': None,
            'train_horizon': 23*7*24,
            'windows_mstl': [13, 17],
            'stl_kwargs_mstl': {'robust': True},
        },
    }
}

## Testing

In [ ]:
do_test = False  # Set to False to skip test

# === Do not edit below ===
if do_test:
    grid = list(product(['F1', 'F2', 'F3', 'F4', 'F5'], ['week', 'day']))
    for id, horizon_type in tqdm(grid, desc="Test", leave=True):

        metadata = {}

        # --- Create directory for test results ---
        timestamp = datetime.now().strftime("%Y%m%dT%H%M%S")
        run_id = f"{id}_{horizon_type}_test_mstl_{timestamp}"
        path = BASE_DIR / "results" / "test" / "mstl" / run_id
        metadata['run_id'] = run_id

        try:
            path.mkdir(parents=True, exist_ok=False)
            logging.info(f"Created directory for test results: {path.relative_to(BASE_DIR)}")


            # --- Set params for cv ---
            out = get_cv_params_for_test(horizon_type)
            metadata['for_cv'] = {
                'step_size': out['step_size'],
                'test_hours': out['test_hours'],
                'end_test_cv': str(out['end_test_actual']),
                'n_windows': out['n_windows'],
                'refit': out['refit'],
                'input_size': None,
            }
            h = 24*7 if horizon_type == 'week' else 24  

            # --- Run cross-validation with the optimal parameters ---
            optimal_params = mstl_optimal_params[id][horizon_type]
            metadata['optimal_params'] = optimal_params
            config = MSTLConfig(**optimal_params)

            heat_id_df = heat_df[heat_df['unique_id'] == id].copy()
            aux_id_df = aux_df[aux_df['unique_id'] == id].copy()
            pipeline = MSTLPipeline(
                target_df=heat_id_df,
                config=config,
            )

            t0 = pd.Timestamp.now()
            cv_df = pipeline.cross_validation(
                h=h, 
                test_size=out['test_hours'],      # Test size in hours
                end_test=out['end_test_actual'],  # End of the test period
                step_size=out['step_size'],       # Step size in hours
                input_size=None,                  # Number of hours to consider for each training
                refit=out['refit'],               # Whether to refit the model for each window
                verbose=True,
            )
            t1 = pd.Timestamp.now()

            avg_elapsed = (t1 - t0).total_seconds() / out['n_fits']
            metadata['avg_el_per_fit'] = avg_elapsed

            cv_df.to_parquet(path / "cv_df.parquet", compression="snappy")

            metadata_path = path / 'metadata.yaml'
            with open(metadata_path, 'w') as f:
                safe_dump_yaml(
                    metadata,
                    f,
                    indent=4, 
                )

            logging.info(f"✓ Artifacts saved successfully for id={id}, horizon={horizon_type}.")

        except KeyboardInterrupt:
            logging.warning("✗ Interrupted; cleaning up.")
            remove_tree(path, require_within=BASE_DIR)
            logging.info("✓ Removed %s", Path(path).relative_to(BASE_DIR) if BASE_DIR in Path(path).resolve().parents else path)
            raise
        except Exception:
            logging.exception("✗ Error during test for id=%s, horizon=%s; cleaning up.", id, horizon_type)
            remove_tree(path, require_within=BASE_DIR)
            logging.info("✓ Removed %s", Path(path).relative_to(BASE_DIR) if BASE_DIR in Path(path).resolve().parents else path)
            raise

    logging.info("✓ Test completed.")

Check and visualize test results:

In [ ]:
# Pick a run_id to analyze
run_id = "F1_day_test_mstl_20251007T194750"

# === Do not edit below ===
# --- Load data ---
path = BASE_DIR / "results" / "test" / "mstl" / run_id
cv_df = pd.read_parquet(path / "cv_df.parquet")
with open(path / "metadata.yaml", "r", encoding="utf-8") as f:
    metadata = yaml.safe_load(f) 

# --- check cv_df ---
from heat_forecast.utils.cv_utils import sanity_cv_df
_ = sanity_cv_df(cv_df, metadata, positive_forecasts=True)

interactive_plot_cutoff_results(
    target_df=heat_df,
    cv_df=cv_df,
    add_context=True,
    figsize=(11, 6)
)